# DCC GARCH Optimization

In [1]:
# import libraries
library(rmgarch)
library(xts)
library(rugarch) 
library(zoo) 

Loading required package: rugarch

Loading required package: parallel


Attaching package: ‘rugarch’


The following object is masked from ‘package:stats’:

    sigma


Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric



Attaching package: ‘xts’


The following objects are masked from ‘package:rmgarch’:

    first, last




read the the transformed data

In [2]:
data <- read.csv("transformed_data.csv")
# split data into training and testing
# Convert data to time series if needed
if(!"Date" %in% colnames(data)) {
    data$Date <- as.Date(rownames(data))
} else {
    # Ensure Date column is in Date format
    data$Date <- as.Date(data$Date)
}

# Calculate split point at 80%
split_point <- floor(nrow(data) * 0.8)

# Split the data
train_data <- data[1:split_point, ]
test_data <- data[(split_point + 1):nrow(data), ]
# Ensure Date columns are properly formatted as Date objects
train_data$Date <- as.Date(train_data$Date)
test_data$Date <- as.Date(test_data$Date)

# Create xts objects
train_data_xts <- xts(train_data[, -which(colnames(train_data) == "Date")], order.by = train_data$Date)
test_data_xts <- xts(test_data[, -which(colnames(test_data) == "Date")], order.by = test_data$Date) 



# Print summary of the split
cat("Training set: ", nrow(train_data), " observations (", round(nrow(train_data)/nrow(data)*100), "%)\n", sep="")
cat("Testing set: ", nrow(test_data), " observations (", round(nrow(test_data)/nrow(data)*100), "%)\n", sep="")

Training set: 3622 observations (80%)
Testing set: 906 observations (20%)


In [4]:
# Check for missing values in training and testing data

# For training data
cat("Missing values in training data:\n")
cat("- Any missing values:", anyNA(train_data_xts), "\n")
cat("- Total missing values:", sum(is.na(train_data_xts)), "\n")
cat("- Missing values by column:\n")
print(colSums(is.na(train_data_xts)))

# For testing data
cat("\nMissing values in testing data:\n")
cat("- Any missing values:", anyNA(test_data_xts), "\n")
cat("- Total missing values:", sum(is.na(test_data_xts)), "\n")
cat("- Missing values by column:\n")
print(colSums(is.na(test_data_xts)))

Missing values in training data:
- Any missing values: FALSE 
- Total missing values: 0 
- Missing values by column:
                 S.P500 Treasury_30y_10y_Spread                     VIX 
                      0                       0                       0 

Missing values in testing data:
- Any missing values: FALSE 
- Total missing values: 0 
- Missing values by column:
                 S.P500 Treasury_30y_10y_Spread                     VIX 
                      0                       0                       0 


In [3]:
head(train_data_xts[, 1])

                S.P500
2007-01-04  0.07250232
2007-01-05 -0.51015312
2007-01-08  0.15121486
2007-01-09 -0.06614069
2007-01-10  0.12900888
2007-01-11  0.47724855

## Univariate GARCH fit

This function (fit_garch_model) is designed to fit a univariate GARCH model to a given time series. It:

• Creates a model specification using given parameters for the ARMA and GARCH orders, distribution, and volatility model.

• Attempts to fit the specified model to the series using a robust solver (with error handling via tryCatch).

• Returns key metrics such as AIC, BIC, HQIC, log-likelihood, convergence status, and the full model summary if the fit is successful or provides an error message if not.

In [3]:
fit_garch_model <- function(series, 
                           arma_order = c(1,1),
                           garch_order = c(1,1),
                           distribution = "std",
                           model = "sGARCH") {
    
    # Create the specification
    spec <- ugarchspec(
        mean.model = list(armaOrder = arma_order),
        variance.model = list(model = model, 
                            garchOrder = garch_order),
        distribution.model = distribution,
        start.pars = list(omega = 0.05, alpha1 = 0.05)
    )
    
    # Fit the model
    fit <- tryCatch({
        ugarchfit(spec, series, solver = "hybrid")
    }, error = function(e) {
        return(list(error = e$message))
    })
    
    # Error handling
    if(is.list(fit) && !is.null(fit$error)) {
        return(list(
            convergence = FALSE,
            error = fit$error,
            aic = NA,
            bic = NA,
            hqic = NA,
            loglik = NA,
            summary = NULL
        ))
    }
    
    # Return results
    list(
        convergence = fit@fit$convergence,
        error = NULL,
        aic = infocriteria(fit)[1],
        bic = infocriteria(fit)[2],
        hqic = infocriteria(fit)[3],
        loglik = likelihood(fit),
        summary = fit
    )
}

In [4]:
# Example run of the function
result <- fit_garch_model(train_data_xts[,1],c(0,0),c(2,1),"sstd","gjrGARCH")
result$summary


*---------------------------------*
*          GARCH Model Fit        *
*---------------------------------*

Conditional Variance Dynamics 	
-----------------------------------
GARCH Model	: gjrGARCH(2,1)
Mean Model	: ARFIMA(0,0,0)
Distribution	: sstd 

Optimal Parameters
------------------------------------
        Estimate  Std. Error   t value Pr(>|t|)
mu      0.013247    0.000071   186.593        0
omega   0.209039    0.000325   642.332        0
alpha1  0.188952    0.000691   273.508        0
alpha2  0.090321    0.000712   126.883        0
beta1   0.892149    0.001031   865.024        0
gamma1 -1.000000    0.000829 -1206.203        0
gamma2 -0.914999    0.001247  -733.587        0
skew    0.938981    0.013823    67.928        0
shape   2.300597    0.013604   169.111        0

Robust Standard Errors:
        Estimate  Std. Error   t value Pr(>|t|)
mu      0.013247    0.000233    56.818        0
omega   0.209039    0.000233   895.263        0
alpha1  0.188952    0.000353   535.259  

This function, optimize_garch_parameters, takes a univariate time series as input and searches for the best GARCH model specification by:

• Setting up a grid of possible ARMA and GARCH parameter combinations along with different distributions and volatility models.  
• Executing the model fitting process in parallel (using multiple cores) to expedite the search.  
• Running the custom fit_garch_model for each parameter setting and collecting key fitting metrics like AIC, BIC, HQIC, and log-likelihood along with the fit-summary



In [5]:
optimize_garch_parameters <- function(series, 
                                    arma_range = list(p = 0:1, q = 0:1),
                                    garch_range = list(p = 1:2, q = 1:2),
                                    distributions = c("norm", "std", "sstd"),
                                    models = c("sGARCH", "eGARCH", "gjrGARCH"),
                                    cores = parallel::detectCores() - 1) {
    
    library(parallel)
    library(doParallel)
    
    # Create cluster
    cl <- makeCluster(cores)
    registerDoParallel(cl)
    
    # Export the fit_garch_model function to workers
    clusterExport(cl, "fit_garch_model")
    
    # Create parameter combinations
    params <- expand.grid(
        arma_p = arma_range$p,
        arma_q = arma_range$q,
        garch_p = garch_range$p,
        garch_q = garch_range$q,
        distribution = distributions,
        model = models)
    
    # Parallel execution
    results <- foreach(i = 1:nrow(params), .packages = c("rmgarch"), .export = "fit_garch_model") %dopar% {
        model_result <- fit_garch_model(
            series = series,
            arma_order = c(params$arma_p[i], params$arma_q[i]),
            garch_order = c(params$garch_p[i], params$garch_q[i]),
            distribution = as.character(params$distribution[i]),
            model = as.character(params$model[i])
        )
        
        # Add parameters to results
        model_result$parameters <- list(
            arma_order = c(params$arma_p[i], params$arma_q[i]),
            garch_order = c(params$garch_p[i], params$garch_q[i]),
            distribution = as.character(params$distribution[i]),
            model = as.character(params$model[i])
        )
        
        # Save the fit model object for later evaluation
        if (!is.null(model_result$summary)) {
            model_result$fit_object <- model_result$summary
        }
                
        model_result
    }
    
    # Stop cluster
    stopCluster(cl)
    
    # Find best model based on AIC
    valid_results <- results[sapply(results, function(x) !is.na(x$aic))]
    best_model <- valid_results[[which.min(sapply(valid_results, function(x) x$aic))]]
    
    list(
        all_results = results,
        best_model = best_model
    )
}

Save all the optimization results along with the best parameters

In [31]:
# Run optimize_garch_parameters for each of the 3 series in train_data_xts
all_optimization_results <- list()
best_models <- list()

# Create a directory for storing model summaries if it doesn't exist
dir.create("model_summaries", showWarnings = FALSE)

for (i in 1:3) {
    cat("Optimizing GARCH parameters for series", i, "\n")
    
    # Run the optimization
    result <- optimize_garch_parameters(
        series = train_data_xts[, i],
        arma_range = list(p = 0:1, q = 0:1),
        garch_range = list(p = 1:2, q = 1:2),
        distributions = c("norm", "std", "sstd"),
        models = c("sGARCH", "eGARCH", "gjrGARCH")
    )
    
    # Store results
    all_optimization_results[[i]] <- result$all_results
    best_models[[i]] <- result$best_model
    
    # Save all model results for this series
    for (j in 1:length(result$all_results)) {
        # Only save models that successfully converged
        if (!is.null(result$all_results[[j]]$fit_object)) {
            model_summary_file <- paste0("model_summaries/series_", i, "_model_", j, ".rds")
            saveRDS(result$all_results[[j]]$fit_object, model_summary_file)
        # Save model summary as text file
        summary_text <- capture.output(print(result$all_results[[j]]$fit_object))
        model_text_file <- paste0("model_summaries/series_", i, "_model_", j, "_summary.txt")
        writeLines(summary_text, model_text_file)
    }
    
    
    }
}



# Create a dataframe with all results for detailed analysis
all_results_df <- do.call(rbind, lapply(1:3, function(i) {
    do.call(rbind, lapply(1:length(all_optimization_results[[i]]), function(j) {
        result <- all_optimization_results[[i]][[j]]
        data.frame(
            Series = i,
            ModelID = j,
            ARMA_P = result$parameters$arma_order[1],
            ARMA_Q = result$parameters$arma_order[2],
            GARCH_P = result$parameters$garch_order[1],
            GARCH_Q = result$parameters$garch_order[2],
            Distribution = result$parameters$distribution,
            Model = result$parameters$model,
            AIC = result$aic,
            BIC = result$bic,
            HQIC = result$hqic,
            LogLikelihood = result$loglik,
            Convergence = !is.null(result$convergence) && result$convergence,
            Error = if(is.null(result$error)) NA else result$error
        )
    }))
}))

# Save all results to CSV and JSON for Python compatibility
write.csv(all_results_df, "garch_optimization_all_results.csv", row.names = FALSE)


Optimizing GARCH parameters for series 1 
Optimizing GARCH parameters for series 2 
Optimizing GARCH parameters for series 3 


saved all the fit summaries which contain the dianostics of the model are saved in the folder "model_summaries". The models will be evaluated based on these summaries using custom scoring function.

# Optimize DCC based on best GARCH parameters

This function, optimize_dcc, performs dynamic conditional correlation (DCC) model estimation on multivariate time series data. It:

• Constructs GARCH specifications for each series using pre-defined parameters.  
• Fits individual GARCH models to each time series with improved solver settings and error handling.  
• Creates a DCC specification by combining the univariate GARCH specifications.  
• Fits the overall DCC model to capture the dynamic correlations between series.  
• Returns fit summary alongwith key fitting metrics (AIC, BIC, HQIC, and LogLikelihood) for model evaluation.

In [3]:
optimize_dcc <- function(data, 
                         dcc_order = c(1,1), 
                         distribution = "mvnorm",
                         garch_params = list(
                             list(arma_order = c(1,0), garch_order = c(2,1), model = "gjrGARCH", dist='sstd'),
                             list(arma_order = c(1,0), garch_order = c(1,1), model = "gjrGARCH", dist='sstd'),
                             list(arma_order = c(1,1), garch_order = c(2,2), model = "gjrGARCH", dist='sstd')
                         )) {
    
    # Create GARCH specifications for each series
    garch_specs <- lapply(garch_params, function(params) {
        ugarchspec(
            mean.model = list(armaOrder = params$arma_order),
            variance.model = list(model = params$model, garchOrder = params$garch_order),
            distribution.model = params$dist
        )
    })
    garch_fits <- list()
    # Fit GARCH models to each series
    garch_fits <- lapply(1:3, function(i) {
        fit <- tryCatch({
            ugarchfit(spec = garch_specs[[i]], data = data[, i], solver = "gosolnp",
                      solver.control = list(tol = 1e-7, max.iter = 5000, trace = 0))
        }, error = function(e) NULL)
        if (is.null(fit)) {
            stop(paste("GARCH fitting failed for series", i))
        }
        return(fit)
    })
    
    # Create DCC specification
    dcc_spec <- dccspec(uspec = multispec(garch_specs), 
                        dccOrder = dcc_order, 
                        distribution = distribution)
    
    # Fit DCC model 
    dcc_fit <- tryCatch({
        dccfit(dcc_spec, data = data,fit=garch_fits, solver = "gosolnp",
               solver.control = list(tol = 1e-7, max.iter = 5000, trace = 0))
    }, error = function(e) NULL)
    
    if (is.null(dcc_fit)) {
        stop("DCC fitting failed.")
    }
    
    # Return results
    list(
        AIC = infocriteria(dcc_fit)[1],
        BIC = infocriteria(dcc_fit)[2],
        HQIC = infocriteria(dcc_fit)[3],
        LogLikelihood = likelihood(dcc_fit),
        fit = dcc_fit
    )
}

In [6]:
# Create a function for DCC optimization with parameterized settings
optimize_dcc_params <- function(data, dcc_order = c(1,1), distribution = "mvnorm") {
    tryCatch({
        return(optimize_dcc(data, dcc_order, distribution))
    }, error = function(e) {
        return(list(
            AIC = NA, 
            BIC = NA, 
            HQIC = NA, 
            LogLikelihood = NA,
            error = as.character(e)
        ))
    })
}

# Set up parameter grid for DCC optimization
library(parallel)
library(doParallel)


# Create parameter combinations
param_grid <- expand.grid(
    a = 1:2,
    b = 1:2,
    distribution = c("mvnorm", "mvt")
)

# Create cluster
num_cores <- parallel::detectCores() - 1
cl <- makeCluster(num_cores)
registerDoParallel(cl)

# Export necessary functions and data to workers
clusterExport(cl, c("optimize_dcc", "train_data_xts"))

# Parallel execution
dcc_results <- foreach(i = 1:nrow(param_grid), .packages = c("rmgarch", "rugarch")) %dopar% {
    dcc_order <- c(param_grid$a[i], param_grid$b[i])
    dist <- as.character(param_grid$distribution[i])
    
    result <- optimize_dcc_params(train_data_xts, dcc_order, dist)
    
    # Add parameters to results
    result$parameters <- list(
        dcc_order = dcc_order,
        distribution = dist
    )
    
    result
}

# Stop cluster
stopCluster(cl)

# Convert results to dataframe for easier analysis
dcc_results_df <- do.call(rbind, lapply(1:length(dcc_results), function(i) {
    result <- dcc_results[[i]]
    data.frame(
        ModelID = i,
        DCC_A = result$parameters$dcc_order[1],
        DCC_B = result$parameters$dcc_order[2],
        Distribution = result$parameters$distribution,
        AIC = result$AIC,
        BIC = result$BIC,
        HQIC = result$HQIC,
        LogLikelihood = result$LogLikelihood,
        Error = if(is.null(result$error)) NA else result$error,
        stringsAsFactors = FALSE
    )
}))

# Find best model based on AIC
best_dcc_model <- dcc_results[[which.min(sapply(dcc_results, function(x) ifelse(is.na(x$AIC), Inf, x$AIC)))]]

# Display results
print(dcc_results_df)
cat("\nBest DCC model parameters:\n")
print(best_dcc_model$parameters)
cat("AIC:", best_dcc_model$AIC, "\n")
cat("BIC:", best_dcc_model$BIC, "\n")

# Save results
write.csv(dcc_results_df, "dcc_optimization_results.csv", row.names = FALSE)

  ModelID DCC_A DCC_B Distribution      AIC      BIC     HQIC LogLikelihood
1       1     1     1       mvnorm       NA       NA       NA            NA
2       2     2     1       mvnorm       NA       NA       NA            NA
3       3     1     2       mvnorm 3.065082 3.126654 3.064887     -5514.864
4       4     2     2       mvnorm       NA       NA       NA            NA
5       5     1     1          mvt       NA       NA       NA            NA
6       6     2     1          mvt       NA       NA       NA            NA
7       7     1     2          mvt       NA       NA       NA            NA
8       8     2     2          mvt 2.788953 2.853945 2.788736     -5012.794
                                                                        Error
1 Error in optimize_dcc(data, dcc_order, distribution): DCC fitting failed.\n
2 Error in optimize_dcc(data, dcc_order, distribution): DCC fitting failed.\n
3                                                                        <NA>
4 Er

In [7]:
dcc_results_df

ModelID,DCC_A,DCC_B,Distribution,AIC,BIC,HQIC,LogLikelihood,Error
<int>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,1,1,mvnorm,NA,NA,NA,NA,"Error in optimize_dcc(data, dcc_order, distribution): DCC fitting failed."
2,2,1,mvnorm,NA,NA,NA,NA,"Error in optimize_dcc(data, dcc_order, distribution): DCC fitting failed."
3,1,2,mvnorm,3.065082,3.126654,3.064887,-5514.864,NA
4,2,2,mvnorm,NA,NA,NA,NA,"Error in optimize_dcc(data, dcc_order, distribution): DCC fitting failed."
5,1,1,mvt,NA,NA,NA,NA,"Error in optimize_dcc(data, dcc_order, distribution): DCC fitting failed."
6,2,1,mvt,NA,NA,NA,NA,"Error in optimize_dcc(data, dcc_order, distribution): DCC fitting failed."
7,1,2,mvt,NA,NA,NA,NA,"Error in optimize_dcc(data, dcc_order, distribution): DCC fitting failed."
8,2,2,mvt,2.788953,2.853945,2.788736,-5012.794,NA
